In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation,Dense,Flatten,BatchNormalization,Conv2D,MaxPooling2D,Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.metrics import binary_accuracy
from sklearn.metrics import confusion_matrix
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import itertools
import skimage
import os
import cv2
import imageio
import random
import glob
import matplotlib.pyplot as plt
%matplotlib inline
import datetime

from tensorflow.keras.applications.xception  import Xception 
from tensorflow.keras.applications.xception  import preprocess_input
#SET SIZE GAMBAR, KENAPA 71x71? SEBAB 200 BERAT SGT FOR CPU SLOW, AND MIN INPUT IS 71 FOR XCEPTION
IMAGE_WIDTH = 71
IMAGE_HEIGHT = 71
IMAGE_SIZE = [71, 71]
#BATCH SIZE SET 1 KALO XDE GPU SO CPU MMG SLOW XLEH BESAR2
BATCH_SIZE_HERE = 1
conv_base = Xception (input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

#SET UNTUK TRANSFER LEARNING (TAK NAK UPDATE WEIGHTS ON PRE-TRAINED LAYERS)
for layer in conv_base.layers:
    layer.trainable = False
    
#UPDATE OPTIONS HERE
namaModel = 'XCEPTION-71'
experimentFolder = 'EXPERIMENT-XCEPTION-71'

OSError: [WinError 8] Not enough memory resources are available to process this command: 'C:\\Users\\user\\anaconda3\\lib\\site-packages\\tensorflow\\python\\distribute\\coordinator'

In [2]:
tambah_augmentation = True
if(tambah_augmentation):
    train_datagen = ImageDataGenerator(
           rescale=1./IMAGE_WIDTH, # resize kepada 71x71
                 rotation_range=0,
                 width_shift_range=0.05, # ubah gamabr kiri kanan 5%
                 height_shift_range=0.05,# ubah gamabr atas bawah 5%
                 zoom_range=0.05, # ubah zoom in/out 5% 
                 horizontal_flip=True, # boleh flip 180 degeree kanan ke kiri
                 vertical_flip= True,# boleh flip 180 degeree atas ke bawah
                 fill_mode='nearest', # isis ruang kosong  style nearest standard
              
        )
else:
    train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./IMAGE_WIDTH) # xde augmentation just resize image 71x71
#set input data folder kat mana untuk training
train_dir = 'CUSTOM_SAMPLE/train' 
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(IMAGE_WIDTH,IMAGE_HEIGHT),# 71x71
                                                    batch_size= BATCH_SIZE_HERE,
                                                    class_mode='categorical', # lebih dari 2 classs so categorical not binary
                                                    classes=["1", "10","11","12"],# nama class yg nak masuk training
                                                    follow_links=True, # default
                                                    shuffle=False)# default

NameError: name 'ImageDataGenerator' is not defined

In [3]:
#validation set xleh augment nnti salah 

val_dir = 'CUSTOM_SAMPLE/test'
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./IMAGE_WIDTH)
validation_generator = test_datagen.flow_from_directory(val_dir,
                                                         target_size=(IMAGE_WIDTH,IMAGE_HEIGHT),# 71x71
                                                        batch_size= BATCH_SIZE_HERE,
                                                        class_mode='categorical',# lebih dari 2 classs so categorical not binary
                                                        classes=["1","10","11","12"],# nama class yg nak masuk training
                                                        follow_links=True,# default
                                                        shuffle=False)# default

NameError: name 'tf' is not defined

In [4]:
import os
import time
currentTimeUnix = str(int(time.time())) 
ExperimentPath = experimentFolder+'-'+currentTimeUnix
os.mkdir(ExperimentPath)


train_steps = int(np.ceil(1919/BATCH_SIZE_HERE)) 
val_steps = int(np.ceil(271  /BATCH_SIZE_HERE)) 
print("Training Steps Per Epcoh: ",train_steps)
print("Validation Steps Per Epcoh: ",val_steps)

NameError: name 'experimentFolder' is not defined

In [5]:
#define model
model = Sequential()
model.add(conv_base)
model.add(Flatten())
model.add(Dense(4, activation = "softmax"))
    
model.compile(Adam(learning_rate=0.001), loss='categorical_crossentropy', 
              metrics=['accuracy'])
model.summary()

NameError: name 'Sequential' is not defined

In [6]:
filepath = ExperimentPath+'/BestLossModel.h5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, patience=2,
                             save_best_only=True, mode='min')

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, 
                                   verbose=1, mode='min', min_lr=0.0001)

#Save log untuk tensorflow tensorboard files untuk plot graph                                
log_dir = ExperimentPath+"/logs/" +namaModel+ datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)                             
callbacks_list = [checkpoint, reduce_lr,tensorboard_callback]

#Start Training and save history
history = model.fit(train_generator, 
                    steps_per_epoch=train_steps, 
                    validation_data=validation_generator,
                    validation_steps=val_steps,
                    epochs=25, 
                    verbose=1,
                    callbacks=callbacks_list,
                    #class_weight=class_weight
                   )

NameError: name 'ExperimentPath' is not defined

In [7]:
from tensorflow.keras.models import load_model
#Svae Checkpoint on last epoch kalo nak sampung bolerh load balik and sambung training 
FinalEpochModel = ExperimentPath+'/LastEpochModel.h5'
model.save(FinalEpochModel)

val_set = validation_generator
# plot the loss
fig=plt.figure()
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history[
    'val_loss'], label='val loss')
plt.legend()
plt.savefig(ExperimentPath+'/Loss.png',dpi=200)

# PLOT ACCURACY FOR ALL REPOCH
fig=plt.figure()
plt.plot(history.history['accuracy'], label='train acc')
plt.plot(history.history['val_accuracy'], label='val acc')
plt.legend()
plt.savefig(ExperimentPath+'/Accuracy.png',dpi=200)

with open(ExperimentPath+'/modelsummary.txt', 'w') as f:

    model.summary(print_fn=lambda x: f.write(x + '\n'))
#RELOAD BEST LOSS MODEL AND RUN TEST
model.load_weights(filepath)
val_loss, val_acc = model.evaluate(val_set)
print('val_loss:', val_loss)
print('val_acc:', val_acc)

predictions_proba = model.predict(val_set, verbose=1)
test_labels = val_set.classes

file1 = open(ExperimentPath+"/report.csv","w")
file2 = open(ExperimentPath+"/report_summary.txt","w")

print("Total Testing Data ",len(val_set.filenames))

NameError: name 'ExperimentPath' is not defined

In [ ]:
VAL_SET_INPUT ="test"

file1.write('No.,Filename,Label,Prediction,Keputusan,\n')

for i in range(len(val_set.filenames)):    
    print(i,",",val_set.filenames[i],",", val_set.classes[i], ",",predictions_proba[i],',',predictions_proba[i].argmax(axis=0) ==  val_set.classes[i])
    # \n is placed to indicate EOL (End of Line)
    file1.write(str(i)+","+val_set.filenames[i]+","+ str(val_set.classes[i])+","+str(predictions_proba[i][0])+","+str(predictions_proba[i][1])+','+str(predictions_proba[i].argmax(axis=0) ==  val_set.classes[i])+'\n')
file1.close() #to change file access modes
file2.close()

In [ ]:
CF_NAME ='';
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[1]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    plt.savefig(ExperimentPath+'/Confusion_Matrix_'+CF_NAME+'.png',dpi=200)
    plt.clf
    
    
def plot_confusion_matrix_Normalized(cm, classes,
                          normalize=True,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[1]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    plt.savefig(ExperimentPath+'/Confusion_Matrix_Normalized_'+CF_NAME+'.png',dpi=200)
    

In [ ]:
# Define the labels of the class indices. These need to match the 
# order shown above.
CF_NAME = 'VAL_BESTLOSSMODEL'

cm = confusion_matrix(val_set.classes,predictions_proba.argmax(axis=1))
cm_plot_labels = [ '1','10','11','12']
plt.clf
plot_confusion_matrix(cm, cm_plot_labels, title='Confusion Matrix')

In [ ]:
# argmax returns the index of the max value in a row

plot_confusion_matrix_Normalized(cm, cm_plot_labels, title='Confusion Matrix-')

In [ ]:
test_filenames = val_set.filenames
# Get the true labels
y_true = val_set.classes
# Get the predicted labels
y_pred = predictions_proba.argmax(axis=1)
from sklearn.metrics import classification_report
# Generate a classification report
report = classification_report(y_true, y_pred, target_names=cm_plot_labels)
print(report)
file3 = open(ExperimentPath+"/CF_SUMMARY.txt","w")
file3.write(report)
file3.close()
